# 台鐵車站的資料分析與視覺化 Part 2. 資料清理

爬蟲的資料很髒，因此要花費大把的心力來處理，先匯入所需套件

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import json
import re

In [3]:
df_wiki = pd.read_json('stationwiki.json')
df_info = pd.read_json('stationinfo.json')

為了方便，把兩張表合併，當時在爬取各車站時，有故意用爬取路線時，車站的名字，這樣可以幫助合併不會出錯

In [4]:
df_info = df_info.merge(df_wiki, on='Station')

現在的資料長這樣：

In [5]:
df_info.head()

,Station,Code,Distance,Level,Link,Open,Line
0,瑞芳（ㄖㄈ）\n,7360\n,0.0\n,一等,/wiki/%E7%91%9E%E8%8A%B3%E7%AB%99,1919年5月5日,宜蘭線、深澳線
1,瑞芳（ㄖㄈ）\n,7360\n,0.0\n,一等,/wiki/%E7%91%9E%E8%8A%B3%E7%AB%99,1919年5月5日,宜蘭線、深澳線
2,瑞芳（ㄖㄈ）\n,7360\n,8.9\n,一等\n,/wiki/%E7%91%9E%E8%8A%B3%E8%BB%8A%E7%AB%99,1919年5月5日,宜蘭線、深澳線
3,瑞芳（ㄖㄈ）\n,7360\n,8.9\n,一等\n,/wiki/%E7%91%9E%E8%8A%B3%E8%BB%8A%E7%AB%99,1919年5月5日,宜蘭線、深澳線
4,海科館\n,7361\n,4.2\n,招呼,/wiki/%E6%B5%B7%E7%A7%91%E9%A4%A8%E7%AB%99,2014年1月9日,深澳線


來統整一下，看了前5筆資料後，目前有的問題：
1. 重複欄位(如第0筆到第3筆資料)：會這樣的原因是因為一個車站會出現在不只一條路線上
2. 中文車站名稱有內部使用的注音符號代碼與換行字元，這些都是我不要的
3. 車站代碼、距離、等級有換行字元

但我不相信問題這麼少，尤其是啟用日期，當時明明有看到很複雜的資料。先放著好了，一步一步來。

## 1. 第一階段整理(解決問題2&3)

問題1我想要留到後面再來處理

In [7]:
df_info['Station'] = df_info.Station.str.replace('\n','')

In [8]:
df_info['Station'] = df_info.Station.str.replace('\（(.+)\）','', regex=True)

In [9]:
df_info['Code'] = df_info.Code.str.replace('\n','')

In [10]:
df_info['Distance'] = df_info.Distance.str.replace('\n','')

In [11]:
df_info['Level'] = df_info.Level.str.replace('\n','')

以上就解決掉2&3了

## 2. 與車站基本資料合併&解決問題1

我想要把車站的資料全部放在一起，這樣比較方便瀏覽

In [12]:
df_basic = pd.read_json('StationBasic.json', dtype=False) #加False是為了讓stationCode不會被轉換成int

In [13]:
df_basic.stationCode

0      0900
1      0910
2      0920
3      0930
4      0940
       ... 
237    7360
238    7361
239    7362
240    7380
241    7390
Name: stationCode, Length: 242, dtype: object

後來想想，因為我們爬取到的資料是"相對"距離，感覺不太好用，所以捨棄。此時我要來處理重複資料了，若用整筆資料完全重複才移除有點不適合，以剛剛看到的瑞芳站為例，前2筆跟後2筆資料的連結是不一樣的，但其實他們都導向同一個頁面，因此我決定只要車站名稱重複就移除。

In [14]:
df_info.drop('Distance', axis=1, inplace=True)
df_info.drop_duplicates(subset='Station', inplace=True)

可以看到我們爬取到的資料筆數筆車站基本資料集的資料還多，這是因為維基百科中，會列出一些目前尚未完工的車站，因此筆數會有出入。

In [15]:
print(len(df_basic), len(df_info))

242 260


In [16]:
df_station = df_basic.merge(df_info, left_on='stationCode', right_on='Code', how='inner')

In [17]:
df_station.shape

(240, 15)

我使用inner join，只取兩邊都有的編碼，這樣才符合我的預期。會發現資料少了兩筆，稍微檢查一下：

In [23]:
for i in df_basic.stationCode:
    if i not in list(df_info.Code):
        print(df_basic[df_basic.stationCode == i]['stationName'])

11    臺北-環島
Name: stationName, dtype: object
169    潮州基地
Name: stationName, dtype: object


這兩筆沒有在我們爬取的資料中非常合理，同理我也回去確認了爬取資料中沒有被合併進來的有哪些，正如前所述，都是尚未完工啟用的車站

但這兩張表合併後有不少重複欄位，我決定要先來處理一下

In [25]:
df_station[df_station.stationName != df_station.Station]

,stationCode,stationName,stationEName,name,ename,stationAddrTw,stationAddrEn,stationTel,gps,Station,Code,Level,Link,Open,Line
57,2210,臺中港,Taichung Port,臺中港,Taichung Port,臺中市清水區頂湳里甲南路 2 號,"No. 2, Jianan Rd., Qingshui Dist., Taichung City",04-26225374,24.30441 120.60231,台中港,2210,二等,/wiki/%E5%8F%B0%E4%B8%AD%E6%B8%AF%E8%BB%8A%E7%...,1922年10月11日,海岸線、台中港線
131,4340,新左營,Xinzuoying,新左營,Xinzuoying,高雄市左營區尾北里站前北路 1 號,"No. 1, Zhanqian N. Rd., Zuoying Dist., Kaohsiu...",07-5887825,22.68721 120.30721,新左營,4340,一等,/wiki/%E6%96%B0%E5%B7%A6%E7%87%9F%E8%BB%8A%E7%...,2006年12月1日,縱貫線
197,7030,新城,Xincheng,新城,Xincheng,花蓮縣新城鄉新城村新興一路 73 號,"No. 73, Xinxing 1st Rd., Xincheng Township, Hu...",03-8611237,24.12761 121.64103,新城(太魯閣),7030,二等,/wiki/%E6%96%B0%E5%9F%8E%EF%BC%88%E5%A4%AA%E9%...,1975年7月26日,北迴線


In [26]:
df_station.drop('Station', axis=1, inplace=True)

In [27]:
df_station[df_station.stationName != df_station.name]

,stationCode,stationName,stationEName,name,ename,stationAddrTw,stationAddrEn,stationTel,gps,Code,Level,Link,Open,Line


In [28]:
df_station.drop('name', axis=1, inplace=True)

In [29]:
df_station[df_station.stationEName != df_station.ename]

,stationCode,stationName,stationEName,ename,stationAddrTw,stationAddrEn,stationTel,gps,Code,Level,Link,Open,Line


In [30]:
df_station.drop('ename', axis=1, inplace=True)

In [31]:
df_station.drop('Code', axis=1, inplace=True)

以上就把所有冗冗的欄位都拿掉了，在拿掉之前，我有先確認那些欄位的差異之處，才敢放心拿掉

## 3. 第二階段整理

前面的整理只是暖暖身而已！這邊才是真正麻煩的地方！

我處理的多半是數值資料，文字資料相對少碰。原則上，"仔細觀察"仍是最重要的原則，找出這些問題的規律，並且一步一步嘗試，直到他變得非常乾淨。

先來看看路線欄位有多髒：

In [32]:
list(df_station.Line)

[' 縱貫線 基隆捷運（規劃中）',
 ' 縱貫線',
 ' 縱貫線、宜蘭線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線\xa0民生汐止線(規劃中)基隆捷運線(規劃中)',
 ' 臺灣鐵路縱貫線  北宜直線鐵路（規劃中） 基隆捷運（規劃中） 臺灣高鐵 板南線',
 ' 臺灣鐵路縱貫線\xa0松山新店線 環狀線（規劃中）',
 ' 臺鐵縱貫線 臺灣高鐵 \xa0淡水信義線  板南線 \xa0機場線',
 ' 縱貫線',
 ' 臺灣鐵路縱貫線 臺灣高鐵 板南線（板橋線） 環狀線',
 '縱貫線\xa0█\xa0泰山板橋輕軌',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線 三鶯線（興建中）三鶯纜車（全案暫緩）',
 ' 縱貫線 綠線（興建中） 棕線（規劃中）',
 ' 縱貫線',
 ' 縱貫線 機場線（興建中）',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線、內灣線',
 ' 內灣線',
 ' 內灣線',
 '內灣線、六家線',
 '六家線',
 ' 內灣線',
 ' 內灣線',
 ' 內灣線',
 ' 內灣線',
 ' 內灣線',
 ' 內灣線',
 ' 內灣線',
 ' 內灣線',
 ' 縱貫線、內灣線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線',
 ' 縱貫線、臺中線、海岸線',
 ' 海岸線',
 ' 海岸線',
 ' 海岸線',
 ' 海岸線',
 ' 海岸線',
 ' 海岸線',
 ' 海岸線',
 ' 海岸線',
 ' 海岸線',
 ' 海岸線',
 ' 海岸線、台中港線',
 ' 海岸線',
 ' 海岸線 \xa0藍線（綜合規劃由交通部審核中）',
 ' 海岸線',
 ' 海岸線',
 ' 海岸線、成追線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線、舊山線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中線',
 ' 臺中

統整一下目前有的問題：
1. 文字前有空格
2. 有\xa0、█等無關訊息
3. 有些路線用空格分隔，有些用頓號，沒有統一
4. 我不想要有<...>這樣的東西

下面的程式碼直接解決上述問題，這就是我喜歡re還有python的地方

In [33]:
df_station['Line'] = df_station.Line.str.strip().replace('\█','', regex=True).replace('\s+', '、',regex=True)\
    .replace('\<(.+)\>','、', regex=True).replace('\、\、','、', regex=True)

但還是會遇到一兩個沒有規則的例外，就當個案處理吧！其實這裡可以不用取代，直接用新的值就好，但有點用上癮了XD

In [152]:
df_station['Line'] = df_station.Line.str.replace('縱貫線\、民生汐止線\(規劃中\)基隆捷運線\(規劃中\)', 
'縱貫線、民生汐止線（規劃中）、基隆捷運線（規劃中）', regex=True).replace('縱貫線\、三鶯線\（興建中\）三鶯纜車\（全案暫緩\）', '縱貫線、三鶯線（興建中）、三鶯纜車（全案暫緩）', regex=True).replace('臺中線海岸線縱貫線南段', '臺中線海岸線、縱貫線南段', regex=True).replace('捷運\：臺南捷運綠線', '臺南捷運綠線', regex=True)

In [34]:
list(df_station.Line)

['縱貫線、基隆捷運（規劃中）',
 '縱貫線',
 '縱貫線、宜蘭線',
 '縱貫線',
 '縱貫線',
 '縱貫線',
 '縱貫線',
 '縱貫線、民生汐止線(規劃中)基隆捷運線(規劃中)',
 '臺灣鐵路縱貫線、北宜直線鐵路（規劃中）、基隆捷運（規劃中）、臺灣高鐵、板南線',
 '臺灣鐵路縱貫線、松山新店線、環狀線（規劃中）',
 '臺鐵縱貫線、臺灣高鐵、淡水信義線、板南線、機場線',
 '縱貫線',
 '臺灣鐵路縱貫線、臺灣高鐵、板南線（板橋線）、環狀線',
 '縱貫線、泰山板橋輕軌',
 '縱貫線',
 '縱貫線',
 '縱貫線',
 '縱貫線、三鶯線（興建中）三鶯纜車（全案暫緩）',
 '縱貫線、綠線（興建中）、棕線（規劃中）',
 '縱貫線',
 '縱貫線、機場線（興建中）',
 '縱貫線',
 '縱貫線',
 '縱貫線',
 '縱貫線',
 '縱貫線',
 '縱貫線',
 '縱貫線',
 '縱貫線',
 '縱貫線、內灣線',
 '內灣線',
 '內灣線',
 '內灣線、六家線',
 '六家線',
 '內灣線',
 '內灣線',
 '內灣線',
 '內灣線',
 '內灣線',
 '內灣線',
 '內灣線',
 '內灣線',
 '縱貫線、內灣線',
 '縱貫線',
 '縱貫線',
 '縱貫線',
 '縱貫線、臺中線、海岸線',
 '海岸線',
 '海岸線',
 '海岸線',
 '海岸線',
 '海岸線',
 '海岸線',
 '海岸線',
 '海岸線',
 '海岸線',
 '海岸線',
 '海岸線、台中港線',
 '海岸線',
 '海岸線、藍線（綜合規劃由交通部審核中）',
 '海岸線',
 '海岸線',
 '海岸線、成追線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線、舊山線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線',
 '臺中線、成追線',
 '臺中線海岸線縱貫線南段',
 '縱貫線南段',
 '縱貫線南段',
 '縱貫線南段',
 '縱貫線南段',
 '縱貫線南段',
 '縱貫線南段',
 '縱貫線南段、集集線',
 '

現在就變得清潔溜溜~ 值得注意的是，這個資料並不符合tidy data的標準，但我只是要方便閱讀而已，因此對我來說不是tidy的沒關係

再來是啟用時間的部分：

In [35]:
list(df_station.Open)

['1891年10月20日',
 '2003年5月9日',
 '縱貫線：1899年7月20日',
 '1891年10月20日',
 '2007年5月8日',
 '1902年6月1日',
 '1891年10月20日',
 '2007年12月30日',
 '臺鐵：1899年7月20日（現今站房：2008年9月21日）臺北捷運：2008年12月25日台灣高鐵：2016年7月1日',
 '臺鐵：1891年10月20日（現今站舍：2008年9月21日）松山新店線：2014年11月15日台北捷運環狀線：可行性研究中',
 '縱貫線：1891年10月20日（現今站房：1989年9月2日）淡水信義線（淡水線）：1997年12月25日板南線（南港線）：1999年12月24日高鐵：2007年3月2日 機場線：2017年3月2日[3]',
 '1901年8月25日（現今站房：1999年7月21日）',
 '臺灣鐵路：1901年8月25日 明治34年(縱貫線)1965年3月5日(臺鐵中和線)1985年(板橋客車場)1999年7月21日(地下化新站)板南線：2006年5月31日台灣高鐵：2007年1月5日環狀線：2020年1月19日(試營運[1][2][3][4])；2020年1月31日(正式啟用[5][6][7])',
 '縱貫線:2011年9月2日',
 '1901年8月25日',
 '2015年12月23日',
 '1903年10月7日',
 '1903年10月7日',
 '縱貫線：1893年10月30日（現今臨時站房：2015年7月26日）',
 '1902年6月1日',
 '縱貫線：1893年10月30日 光緒19年',
 '1900年4月10日',
 '1893年（光緒19年）10月30日',
 '1929年10月1日',
 '2017年9月6日',
 '2012年9月28日',
 '1929年',
 '1893年11月30日',
 '1897年（明治30年）9月10日',
 '2011年11月11日',
 '2011年11月11日',
 '2011年11月11日',
 '1947年11月5日',
 '2011年11月11日',
 '1970年2月6日',
 '2001年11月24日',
 '1947年11月5日',
 '1950年12月27日',
 '1950年12月27日',

比剛剛髒了非常多，好像沒有規律，但冷靜下來可以發現：我們要的啟用時間都是第一個日期。我不在意其他路線或站房的啟用日期，我只要這個車站本身第一次啟用的時間就好了。另外有些年代久遠的還有清領或日治時期的年號，這個也要一併去除。

In [36]:
df_station['Open'] = df_station.Open.str.replace('日(.+)', '日', regex=True).replace('(.+)\：', '', regex=True).replace('(.+):','', regex=True).replace('(.+)\-', '', regex=True).replace('\n(.+)','', regex=True).replace('\（(.+)\）', '', regex=True)

In [37]:
list(df_station.Open)

['1891年10月20日',
 '2003年5月9日',
 '1899年7月20日',
 '1891年10月20日',
 '2007年5月8日',
 '1902年6月1日',
 '1891年10月20日',
 '2007年12月30日',
 '1899年7月20日',
 '1891年10月20日',
 '1891年10月20日',
 '1901年8月25日',
 '1901年8月25日',
 '2011年9月2日',
 '1901年8月25日',
 '2015年12月23日',
 '1903年10月7日',
 '1903年10月7日',
 '1893年10月30日',
 '1902年6月1日',
 '1893年10月30日',
 '1900年4月10日',
 '1893年10月30日',
 '1929年10月1日',
 '2017年9月6日',
 '2012年9月28日',
 '1929年',
 '1893年11月30日',
 '1897年9月10日',
 '2011年11月11日',
 '2011年11月11日',
 '2011年11月11日',
 '1947年11月5日',
 '2011年11月11日',
 '1970年2月6日',
 '2001年11月24日',
 '1947年11月5日',
 '1950年12月27日',
 '1950年12月27日',
 '1950年12月27日',
 '1962年11月28日',
 '1951年9月11日',
 '1893年10月30日',
 '2016年6月29日',
 '1902年8月10日',
 '1928年3月12日',
 '1902年8月10日',
 '1922年10月11日',
 '1922年10月11日',
 '1922年10月11日',
 '1922年10月11日',
 '1922年10月11日',
 '1922年10月11日',
 '1922年10月11日',
 '1922年10月11日',
 '1922年10月11日',
 '1922年10月11日',
 '1922年10月11日',
 '1920年12月25日',
 '1920年12月25日',
 '1920年12月25日',
 '1920年12月25日',
 '1922年10月11日',
 '1903年5月25日',
 '1903年5月25日',


看起來舒服多了，接下來就是去除空格，把年月日換成斜線，比較好在表格中轉換為日期時間格式

In [38]:
df_station['Open'] = df_station.Open.str.strip()

In [39]:
df_station['Open'] = df_station.Open.str.replace('年','/').replace('月','/', regex=True).replace('日','', regex=True)

另外有一個車站只有年份沒有日期，因此我上網找資料將他補上：

In [40]:
df_station.loc[df_station.Open == '1929/', 'Open'] = '1929/11/1'

In [41]:
list(df_station.Open)

['1891/10/20',
 '2003/5/9',
 '1899/7/20',
 '1891/10/20',
 '2007/5/8',
 '1902/6/1',
 '1891/10/20',
 '2007/12/30',
 '1899/7/20',
 '1891/10/20',
 '1891/10/20',
 '1901/8/25',
 '1901/8/25',
 '2011/9/2',
 '1901/8/25',
 '2015/12/23',
 '1903/10/7',
 '1903/10/7',
 '1893/10/30',
 '1902/6/1',
 '1893/10/30',
 '1900/4/10',
 '1893/10/30',
 '1929/10/1',
 '2017/9/6',
 '2012/9/28',
 '1929/11/1',
 '1893/11/30',
 '1897/9/10',
 '2011/11/11',
 '2011/11/11',
 '2011/11/11',
 '1947/11/5',
 '2011/11/11',
 '1970/2/6',
 '2001/11/24',
 '1947/11/5',
 '1950/12/27',
 '1950/12/27',
 '1950/12/27',
 '1962/11/28',
 '1951/9/11',
 '1893/10/30',
 '2016/6/29',
 '1902/8/10',
 '1928/3/12',
 '1902/8/10',
 '1922/10/11',
 '1922/10/11',
 '1922/10/11',
 '1922/10/11',
 '1922/10/11',
 '1922/10/11',
 '1922/10/11',
 '1922/10/11',
 '1922/10/11',
 '1922/10/11',
 '1922/10/11',
 '1920/12/25',
 '1920/12/25',
 '1920/12/25',
 '1920/12/25',
 '1922/10/11',
 '1903/5/25',
 '1903/5/25',
 '1903/5/25',
 '1903/7/1',
 '1903/10/7',
 '1903/10/7',
 '199

超級乾淨的！接下來就是轉換成內建的日期時間格式，這能讓我比較好對時間做運算(如果有需求的話)

In [42]:
df_station['Open'] = pd.to_datetime(df_station.Open)

In [43]:
df_station.dtypes

stationCode              object
stationName              object
stationEName             object
stationAddrTw            object
stationAddrEn            object
stationTel               object
gps                      object
Level                    object
Link                     object
Open             datetime64[ns]
Line                     object
dtype: object

至此處理完成，當然我有確認其他欄位沒有任何問題才敢下此結論。存成.csv跟.json檔以防不時之需

In [44]:
df_station.to_csv('df_station.csv')

In [45]:
import json

In [46]:
df_station = pd.read_csv('df_station.csv', index_col=0)

In [47]:
df_station.to_json('df_station.json', orient='records')

In [48]:
with open(r"df_station.json", "r") as read_file:
    data2 = json.load(read_file)

In [49]:
data2

[{'stationCode': 900,
  'stationName': '基隆',
  'stationEName': 'Keelung',
  'stationAddrTw': '基隆市中山區民治里1鄰中山一路16之 1號',
  'stationAddrEn': 'No. 16-1, Zhongshan 1st Rd, Zhongshan Dist., Keelung City',
  'stationTel': '02-24263743',
  'gps': '25.13401 121.74013',
  'Level': '一等',
  'Link': '/wiki/%E5%9F%BA%E9%9A%86%E8%BB%8A%E7%AB%99',
  'Open': '1891-10-20',
  'Line': '縱貫線、基隆捷運（規劃中）'},
 {'stationCode': 910,
  'stationName': '三坑',
  'stationEName': 'Sankeng',
  'stationAddrTw': '基隆市仁愛區德厚里龍安街 206 號',
  'stationAddrEn': 'No. 206, Long’an St., Ren`ai Dist., Keelung City',
  'stationTel': '02-24230289',
  'gps': '25.12305 121.74202',
  'Level': '簡易',
  'Link': '/wiki/%E4%B8%89%E5%9D%91%E8%BB%8A%E7%AB%99',
  'Open': '2003-05-09',
  'Line': '縱貫線'},
 {'stationCode': 920,
  'stationName': '八堵',
  'stationEName': 'Badu',
  'stationAddrTw': '基隆市暖暖區八南里八堵路 142 號',
  'stationAddrEn': 'No. 142, Badu Rd., Nuannuan Dist., Keelung City',
  'stationTel': '02-24560841',
  'gps': '25.10843 121.72898',
  'Level